# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,castro,-24.7911,-50.0119,21.52,85,60,4.12,BR,1675539441
1,1,kolpashevo,58.3131,82.9089,-19.35,100,100,0.90,RU,1675539442
2,2,umea,63.8284,20.2597,-11.16,77,0,2.57,SE,1675539442
3,3,rikitea,-23.1203,-134.9692,25.85,76,3,7.04,PF,1675539303
4,4,hasaki,35.7333,140.8333,2.90,56,2,7.69,JP,1675539442


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.8,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400
    
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[
    (city_data_df["Max Temp"] < 29) & (city_data_df["Max Temp"] > 20) \
    & (city_data_df["Wind Speed"] < 10) \
    & (city_data_df["Cloudiness"] < 10)
]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,rikitea,-23.1203,-134.9692,25.85,76,3,7.04,PF,1675539303
9,9,bredasdorp,-34.5322,20.0403,22.04,81,0,1.78,ZA,1675539301
34,34,pisco,-13.7000,-76.2167,24.03,83,0,4.63,PE,1675539176
46,46,miraflores,19.2167,-98.8083,22.21,28,0,1.57,MX,1675539453
47,47,port elizabeth,-33.9180,25.5701,24.06,88,0,2.06,ZA,1675539453
52,52,caravelas,-17.7125,-39.2481,28.26,71,7,4.98,BR,1675539455
60,60,todos santos,23.4500,-110.2167,27.39,67,5,2.78,MX,1675539456
115,115,mar del plata,-38.0023,-57.5575,24.35,65,0,7.72,AR,1675538776
145,145,cabo san lucas,22.8909,-109.9124,27.61,49,0,3.60,MX,1675539179
147,147,carnarvon,-24.8667,113.6333,25.65,72,2,1.63,AU,1675539319


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[["City","Country","Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,rikitea,PF,-23.1203,-134.9692,76,
9,bredasdorp,ZA,-34.5322,20.0403,81,
34,pisco,PE,-13.7000,-76.2167,83,
46,miraflores,MX,19.2167,-98.8083,28,
47,port elizabeth,ZA,-33.9180,25.5701,88,
52,caravelas,BR,-17.7125,-39.2481,71,
60,todos santos,MX,23.4500,-110.2167,67,
115,mar del plata,AR,-38.0023,-57.5575,65,
145,cabo san lucas,MX,22.8909,-109.9124,49,
147,carnarvon,AU,-24.8667,113.6333,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rikitea - nearest hotel: Chez Bianca & Benoit
bredasdorp - nearest hotel: Victoria Hotel
pisco - nearest hotel: La Portada
miraflores - nearest hotel: Hotel Hacienda Panoaya
port elizabeth - nearest hotel: Waterford Hotel
caravelas - nearest hotel: Pousada dos Navegantes
todos santos - nearest hotel: Hotel Casa Tota
mar del plata - nearest hotel: Nuevo Ostende
cabo san lucas - nearest hotel: Comfort Rooms
carnarvon - nearest hotel: No hotel found
chuy - nearest hotel: Alerces
porto novo - nearest hotel: Hôtel 6500 CFA
morondava - nearest hotel: Hotel Sharon Menabe
copiapo - nearest hotel: Hotel Il Riposo
tolaga bay - nearest hotel: No hotel found
matamoros - nearest hotel: No hotel found
saint pete beach - nearest hotel: Grand Plaza Hotel
belmonte - nearest hotel: No hotel found
calvinia - nearest hotel: No hotel found
richards bay - nearest hotel: Protea Hotel
nueva helvecia - nearest hotel: Hotel del Prado
puri - nearest hotel: Divine Beach Homestay
sur - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
3,rikitea,PF,-23.1203,-134.9692,76,Chez Bianca & Benoit
9,bredasdorp,ZA,-34.5322,20.0403,81,Victoria Hotel
34,pisco,PE,-13.7000,-76.2167,83,La Portada
46,miraflores,MX,19.2167,-98.8083,28,Hotel Hacienda Panoaya
47,port elizabeth,ZA,-33.9180,25.5701,88,Waterford Hotel
52,caravelas,BR,-17.7125,-39.2481,71,Pousada dos Navegantes
60,todos santos,MX,23.4500,-110.2167,67,Hotel Casa Tota
115,mar del plata,AR,-38.0023,-57.5575,65,Nuevo Ostende
145,cabo san lucas,MX,22.8909,-109.9124,49,Comfort Rooms
147,carnarvon,AU,-24.8667,113.6333,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    alpha = 0.8,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    hover_cols = ["Hotel Name", "Country"]
    
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)